# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f4260e4a070>
├── 'a' --> tensor([[ 1.5866, -0.5300, -1.0220],
│                   [ 2.1208,  1.0100,  0.5656]])
└── 'x' --> <FastTreeValue 0x7f4260e8bf10>
    └── 'c' --> tensor([[-1.9281,  1.5287,  1.7789, -0.3805],
                        [-0.2391, -0.6947, -1.4315,  0.5217],
                        [-0.4571, -0.9997, -0.6935, -0.7640]])

In [4]:
t.a

tensor([[ 1.5866, -0.5300, -1.0220],
        [ 2.1208,  1.0100,  0.5656]])

In [5]:
%timeit t.a

61.7 ns ± 0.0491 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f4260e4a070>
├── 'a' --> tensor([[ 0.9975, -0.8112,  2.1055],
│                   [-1.2894,  1.0562, -0.4260]])
└── 'x' --> <FastTreeValue 0x7f4260e8bf10>
    └── 'c' --> tensor([[-1.9281,  1.5287,  1.7789, -0.3805],
                        [-0.2391, -0.6947, -1.4315,  0.5217],
                        [-0.4571, -0.9997, -0.6935, -0.7640]])

In [7]:
%timeit t.a = new_value

68.5 ns ± 0.0281 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.5866, -0.5300, -1.0220],
               [ 2.1208,  1.0100,  0.5656]]),
    x: Batch(
           c: tensor([[-1.9281,  1.5287,  1.7789, -0.3805],
                      [-0.2391, -0.6947, -1.4315,  0.5217],
                      [-0.4571, -0.9997, -0.6935, -0.7640]]),
       ),
)

In [10]:
b.a

tensor([[ 1.5866, -0.5300, -1.0220],
        [ 2.1208,  1.0100,  0.5656]])

In [11]:
%timeit b.a

52.4 ns ± 0.0817 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.8699,  1.2207, -2.0784],
               [ 1.3050, -1.3136, -0.8296]]),
    x: Batch(
           c: tensor([[-1.9281,  1.5287,  1.7789, -0.3805],
                      [-0.2391, -0.6947, -1.4315,  0.5217],
                      [-0.4571, -0.9997, -0.6935, -0.7640]]),
       ),
)

In [13]:
%timeit b.a = new_value

484 ns ± 0.172 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

877 ns ± 7.59 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 30.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

134 µs ± 297 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

136 µs ± 586 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f41b5ad1c40>
├── 'a' --> tensor([[[ 1.5866, -0.5300, -1.0220],
│                    [ 2.1208,  1.0100,  0.5656]],
│           
│                   [[ 1.5866, -0.5300, -1.0220],
│                    [ 2.1208,  1.0100,  0.5656]],
│           
│                   [[ 1.5866, -0.5300, -1.0220],
│                    [ 2.1208,  1.0100,  0.5656]],
│           
│                   [[ 1.5866, -0.5300, -1.0220],
│                    [ 2.1208,  1.0100,  0.5656]],
│           
│                   [[ 1.5866, -0.5300, -1.0220],
│                    [ 2.1208,  1.0100,  0.5656]],
│           
│                   [[ 1.5866, -0.5300, -1.0220],
│                    [ 2.1208,  1.0100,  0.5656]],
│           
│                   [[ 1.5866, -0.5300, -1.0220],
│                    [ 2.1208,  1.0100,  0.5656]],
│           
│                   [[ 1.5866, -0.5300, -1.0220],
│                    [ 2.1208,  1.0100,  0.5656]]])
└── 'x' --> <FastTreeValue 0x7f4260e79e80>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.7 µs ± 38.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f41b5ad1ac0>
├── 'a' --> tensor([[ 1.5866, -0.5300, -1.0220],
│                   [ 2.1208,  1.0100,  0.5656],
│                   [ 1.5866, -0.5300, -1.0220],
│                   [ 2.1208,  1.0100,  0.5656],
│                   [ 1.5866, -0.5300, -1.0220],
│                   [ 2.1208,  1.0100,  0.5656],
│                   [ 1.5866, -0.5300, -1.0220],
│                   [ 2.1208,  1.0100,  0.5656],
│                   [ 1.5866, -0.5300, -1.0220],
│                   [ 2.1208,  1.0100,  0.5656],
│                   [ 1.5866, -0.5300, -1.0220],
│                   [ 2.1208,  1.0100,  0.5656],
│                   [ 1.5866, -0.5300, -1.0220],
│                   [ 2.1208,  1.0100,  0.5656],
│                   [ 1.5866, -0.5300, -1.0220],
│                   [ 2.1208,  1.0100,  0.5656]])
└── 'x' --> <FastTreeValue 0x7f41b6247070>
    └── 'c' --> tensor([[-1.9281,  1.5287,  1.7789, -0.3805],
                        [-0.2391, -0.6947, -1.4315,  0.5217],
                 

In [23]:
%timeit t_cat(trees)

30.3 µs ± 67.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.5 µs ± 58.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.5866, -0.5300, -1.0220],
                [ 2.1208,  1.0100,  0.5656]],
       
               [[ 1.5866, -0.5300, -1.0220],
                [ 2.1208,  1.0100,  0.5656]],
       
               [[ 1.5866, -0.5300, -1.0220],
                [ 2.1208,  1.0100,  0.5656]],
       
               [[ 1.5866, -0.5300, -1.0220],
                [ 2.1208,  1.0100,  0.5656]],
       
               [[ 1.5866, -0.5300, -1.0220],
                [ 2.1208,  1.0100,  0.5656]],
       
               [[ 1.5866, -0.5300, -1.0220],
                [ 2.1208,  1.0100,  0.5656]],
       
               [[ 1.5866, -0.5300, -1.0220],
                [ 2.1208,  1.0100,  0.5656]],
       
               [[ 1.5866, -0.5300, -1.0220],
                [ 2.1208,  1.0100,  0.5656]]]),
    x: Batch(
           c: tensor([[[-1.9281,  1.5287,  1.7789, -0.3805],
                       [-0.2391, -0.6947, -1.4315,  0.5217],
                       [-0.4571, -0.9997, -0.6935, -0.7640]],
         

In [26]:
%timeit Batch.stack(batches)

77.3 µs ± 246 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.5866, -0.5300, -1.0220],
               [ 2.1208,  1.0100,  0.5656],
               [ 1.5866, -0.5300, -1.0220],
               [ 2.1208,  1.0100,  0.5656],
               [ 1.5866, -0.5300, -1.0220],
               [ 2.1208,  1.0100,  0.5656],
               [ 1.5866, -0.5300, -1.0220],
               [ 2.1208,  1.0100,  0.5656],
               [ 1.5866, -0.5300, -1.0220],
               [ 2.1208,  1.0100,  0.5656],
               [ 1.5866, -0.5300, -1.0220],
               [ 2.1208,  1.0100,  0.5656],
               [ 1.5866, -0.5300, -1.0220],
               [ 2.1208,  1.0100,  0.5656],
               [ 1.5866, -0.5300, -1.0220],
               [ 2.1208,  1.0100,  0.5656]]),
    x: Batch(
           c: tensor([[-1.9281,  1.5287,  1.7789, -0.3805],
                      [-0.2391, -0.6947, -1.4315,  0.5217],
                      [-0.4571, -0.9997, -0.6935, -0.7640],
                      [-1.9281,  1.5287,  1.7789, -0.3805],
                      [-0.2391, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 388 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

312 µs ± 571 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
